# Analysing Drift Scenarios
Quantify how configured drift windows impact downstream metrics.

In [ ]:
import pandas as pd

from segf import DriftScenario, SegfConfig, SegfValidator, SyntheticEntityGraphForge, TargetStats

config = SegfConfig(
    drift_scenarios=[
        DriftScenario(name="holiday_spike", start_day=15, end_day=25, fraud_multiplier=1.5, chargeback_multiplier=2.2),
        DriftScenario(name="post_holiday_lull", start_day=26, end_day=40, fraud_multiplier=0.6, legit_multiplier=0.7),
    ]
)
forge = SyntheticEntityGraphForge(config)
result = forge.generate()

baseline_validator = SegfValidator(
    TargetStats(
        expected_user_fraud_ratio=config.population.fraud_user_ratio,
        expected_chargeback_rate=config.events.chargeback_prob_fraud * 0.5,
        expected_daily_transactions=config.events.daily_txn_rate_legit,
        drift_windows={
            "holiday_spike": {"chargeback_multiplier": 2.2},
            "post_holiday_lull": {"txn_multiplier": 0.7},
        },
        tolerance=0.15,
    )
)
report = baseline_validator.evaluate(users=result.users, events=result.events, lifecycles=result.lifecycles)
pd.DataFrame(report.drift_metrics).T


The resulting frame surfaces multipliers against the baseline rates for each named drift window.